In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

In [10]:
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
# COHERE_API_KEY

In [11]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

cache_store = LocalFileStore("./cache/")

In [12]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2401.55it/s]


In [13]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
pretty_print_docs(chunked)

Document 1:

The Quest for Tyr is the second main quest you’ll receive in God of War Ragnarok . This walkthrough guide will offer tips for conquering enemies and defeating challenging bosses, solutions to all mine and water trough puzzles, and locations of all collectibles that Kratos and Atreus will encounter along their journey through Svartalfheim as they attempt to track down Tyr, the son of Odin.
Spoiler Warning This walkthrough will include story spoilers for the second quest, The Quest for Tyr, so proceed with caution.
The Quest for Tyr Walkthrough Guide
Journey Rewards : Tap to Reveal .
Need help taking down one of the many bosses you’ll encounter throughout this chapter? Click the links below to jump to…
How to Defeat Dreki How to Defeat Bergsra
Stuck and need some help with a particular objective or tricky puzzle mine puzzle in The Quest for Tyr? Click the spoiler box below to find a list of links to all objectives in this chapter.
Show Spoilers
Journey to Nidavellir
Having a

In [14]:
embeddings_model = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

In [15]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace=embeddings_model.model)

In [16]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 378 ms, sys: 30 ms, total: 408 ms
Wall time: 2.12 s


In [ ]:
llm = Cohere(temperature=2)

In [59]:
retriever = db.as_retriever(search_type="mmr")
docs = retriever.get_relevant_documents("How do I deal with Enhanced Grims?")
pretty_print_docs(docs)

Document 1:

Next, climb up the ledge to your right and burn the Bramble blocking the path, where you’ll then find a Hacksilver pouch by the ledge, and a Healthstone and Hacksilver chest along the bridge.

With those collected, return to Atreus and burn the Bramble above, along the edges of the rock formation. Climb up to the top of the formation, where you’ll find a Red Coffin with +2 Rawhide and some Hacksilver. You can also find a Hacksilver pouch hidden beneath some Bramble, next to a fallen Grim.
Furthermore, drop down the nearby ledge and destroy the hanging bucket to receive +1 Rawhide. With that collected, return to your boat and continue slightly further up the river, where you’ll discover another docking location – once docked, another group of Wretches and Grims’ll attack you, so take them out to continue.

Once the area is clear, head over towards Atreus and interact with the contraption that’s stopping a geyser from spouting. With the geysers free, take the path to your ri

In [60]:
llm = Cohere(temperature=2)

In [61]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# compressed_docs = compression_retriever.get_relevant_documents(
#     "How do I deal with Enhanced Grims?"
# )
# pretty_print_docs(compressed_docs)

In [62]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)
chain({"query": "How do I deal with enhanced grims in God of War Ragnarok?"})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': 'How do I deal with enhanced grims in God of War Ragnarok?',
 'result': "To deal with enhanced Grims in God of War Ragnarok, you need to take them out when reaching the top of a formation and continue moving around the area as they toss poisonous projectiles and climb high up onto rock formations. They also leave poisonous pools on the ground that slowly deplete your health when stepped on. Additionally, throwing the Leviathan Axe at an Ormr can be an effective strategy since they move quickly. As you progress through the game, you'll encounter Cursed Gims and Kvasir Poems to find scattered about different realms. In the city of Nidavellir, it's recommended to track down someone who will speak to you at the Tavern mentioned by Sindri before heading towards the large statue of Odin."}

In [63]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "How do I deal with Enhanced Grims?"
)
pretty_print_docs(compressed_docs)

Document 1:

Next, climb up the ledge to your right and burn the Bramble blocking the path, where you’ll then find a Hacksilver pouch by the ledge, and a Healthstone and Hacksilver chest along the bridge.

With those collected, return to Atreus and burn the Bramble above, along the edges of the rock formation. Climb up to the top of the formation, where you’ll find a Red Coffin with +2 Rawhide and some Hacksilver. You can also find a Hacksilver pouch hidden beneath some Bramble, next to a fallen Grim.
Furthermore, drop down the nearby ledge and destroy the hanging bucket to receive +1 Rawhide. With that collected, return to your boat and continue slightly further up the river, where you’ll discover another docking location – once docked, another group of Wretches and Grims’ll attack you, so take them out to continue.

Once the area is clear, head over towards Atreus and interact with the contraption that’s stopping a geyser from spouting. With the geysers free, take the path to your ri

In [65]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)
chain({"query": "How do I deal with Enhanced Grims in God of War Ragnarok?"})

{'query': 'How do I deal with Enhanced Grims in God of War Ragnarok?',
 'result': "Enhanced Grims are stronger versions of the regular Grim enemies found in God of War Ragnarok. To deal with them, here are some tips:\n\n1. Focus on their weak points: Enhanced Grims have a weak spot on their back that deals more damage when hit. Try to aim for this area whenever possible.\n\n2. Use ranged attacks: The Leviathan Axe's Ranged Skills, such as Freezing Throw I and Axe Throw, can help you take out Enhanced Grims from a distance before they get too close. This will give you more time to dodge their attacks.\n\n3. Parry their attacks: Just like with regular Grims, you can parry Enhanced Grims' attacks by timing your shield block at the right moment. This not only avoids damage but also stuns them briefly, giving you a chance to attack.\n\n4. Use Mjolnir: If you have the Mjolnir hammer from the previous game, it can be very effective against Enhanced Grims. The hammer's charged attacks deal mas

In [ ]:
query = "How do I deal with Enhanced Grims?"
search_result = db.similarity_search(query)
# pretty_print_docs(search_result)
# search_result[0].page_content